En esta clase, aprenderá cómo utilizar(pipelines) canalizaciones para limpiar su código de modelado.

# Introducción

Las canalizaciones son una forma sencilla de mantener organizado el código de modelado y preprocesamiento de datos. Específicamente, una canalización agrupa pasos de preprocesamiento y modelado para que pueda utilizar todo el paquete como si fuera un solo paso.

Muchos científicos de datos combinan modelos sin canalizaciones, pero las canalizaciones tienen algunos beneficios importantes. Estos incluyen:

1. Código más limpio: la contabilidad de los datos en cada paso del preprocesamiento puede resultar complicada. Con una canalización, no necesitará realizar un seguimiento manual de sus datos de capacitación y validación en cada paso.

2. Menos errores: hay menos oportunidades de aplicar mal un paso u olvidar un paso de preprocesamiento.

3. Más fácil de producir: puede resultar sorprendentemente difícil hacer la transición de un modelo de un prototipo a algo que se pueda implementar a escala. No entraremos aquí en las muchas preocupaciones relacionadas, pero los oleoductos pueden ayudar.

4. Más opciones para la validación del modelo: verá un ejemplo en el siguiente tutorial, que cubre la validación cruzada.

# Ejemplo

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento con el método head() a continuación. Observe que los datos contienen datos categóricos y columnas con valores faltantes. ¡Con una tubería, es fácil lidiar con ambos!

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


Construimos el pipeline completo en tres pasos.

### Paso 1: Definir los pasos de preprocesamiento

De manera similar a cómo una canalización agrupa los pasos de preprocesamiento y modelado, usamos la clase ColumnTransformer para agrupar diferentes pasos de preprocesamiento. El siguiente código:

* imputa valores faltantes en datos numéricos, y

* imputa valores faltantes y aplica una codificación one-hot a datos categóricos.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### Paso 2: Definir el modelo

A continuación, definimos un modelo de bosque aleatorio con la conocida clase RandomForestRegressor.

In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Paso 3: Crear y evaluar la canalización

Finalmente, usamos la clase Pipeline para definir una canalización que agrupa los pasos de preprocesamiento y modelado. Hay algunas cosas importantes a tener en cuenta:

* Con el pipeline, preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código. (Por el contrario, sin una canalización, tenemos que realizar la imputación, la codificación one-hot y el entrenamiento del modelo en pasos separados. ¡Esto se vuelve especialmente complicado si tenemos que lidiar con variables numéricas y categóricas!)

* Con la canalización, suministramos las características no procesadas en X_valid al comando predict() y la canalización preprocesa automáticamente las características antes de generar predicciones. (Sin embargo, sin una canalización, debemos recordar preprocesar los datos de validación antes de hacer predicciones).

In [6]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


### Conclusión

Las canalizaciones son valiosas para limpiar el código de aprendizaje automático y evitar errores, y son especialmente útiles para flujos de trabajo con preprocesamiento de datos sofisticado.